In [ ]:
from google.colab import auth
auth.authenticate_user()
import gspread
from sklearn.metrics import plot_confusion_matrix, accuracy_score
from oauth2client.client import GoogleCredentials
from gspread_dataframe import set_with_dataframe

gc = gspread.authorize(GoogleCredentials.get_application_default())

KeyboardInterrupt: ignored

In [ ]:
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
import time 
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier

def load_data():
  path1 = '/content/drive/My Drive/TDT4173 Maskinlæring /data_batch_1'
  path2 = '/content/drive/My Drive/TDT4173 Maskinlæring /data_batch_2'
  path3 = '/content/drive/My Drive/TDT4173 Maskinlæring /data_batch_3'
  path4 = '/content/drive/My Drive/TDT4173 Maskinlæring /data_batch_4'  
  path5 = '/content/drive/My Drive/TDT4173 Maskinlæring /data_batch_5'
  path6 = '/content/drive/My Drive/TDT4173 Maskinlæring /test_batch'

  listOfTestFiles = [path1, path2, path3, path4, path5, path6]

  train = []
  train_labels = []
  test = []
  test_labels = []

  #For collecting Training data:
  for file in listOfTestFiles[0:5]:
      with open(file,'rb') as fo:
          dict = pickle.load(fo,encoding='bytes')
          train.append(dict[b'data'])
          train_labels.append(dict[b'labels'])

  #for collecting Testing data
  with open(path6,'rb') as fo:
          dict = pickle.load(fo,encoding='bytes')
          test.append(dict[b'data'])
          test_labels.append(dict[b'labels']) 

  dictData = {}
  dictData['train_data'] = np.reshape(np.array(train),newshape=(np.array(train).shape[0]*np.array(train).shape[1],np.array(train).shape[2]))
  dictData['train_labels'] = np.reshape(np.array(train_labels),newshape=(np.array(train_labels).shape[0]*np.array(train_labels).shape[1]))
  dictData['test_data'] = np.reshape(np.array(test),newshape=(np.array(test).shape[0]*np.array(test).shape[1],np.array(test).shape[2]))
  dictData['test_labels'] = np.reshape(np.array(test_labels),newshape=(np.array(test_labels).shape[0]*np.array(test_labels).shape[1]))
  return dictData

In [ ]:
#Run pca to find number of components d that have >p variance
def find_components(p, train_x):
  starttime = time.time()
  pca = PCA()
  pca.fit(train_x)
  cum_sum = np.cumsum(pca.explained_variance_ratio_, axis = 0)
  out = np.cumsum(pca.explained_variance_ratio_)
  top_percentile = np.where(cum_sum>p)[0][0]
  print("Top ",p*100, "% is gained by choosing: ", top_percentile, "components. Time used: ", time.time()-starttime)
  return top_percentile

In [ ]:
plt.figure()
plt.plot(col[:1024])
plt.plot(gray)
plt.legend(['Colour', 'Gray'], loc='center right')
plt.ylabel('Accumulated explained variance ratio')
plt.xlabel("Number of features")
plt.title("Accumulated EVR plot")
plt.show()

In [ ]:
#View image
def view(image):
  temp = image
  if (image.shape[0]==3072): #For RGB images
    R = temp[0:1024].reshape(32,32)
    G = np.reshape(temp[1024:2048],newshape=(32,32))
    B = np.reshape(temp[2048:],newshape=(32,32))
    temp = np.dstack((R,G,B))   #for stacking all these 32,32 matrices.
    plt.imshow(temp)
  else: #For grayscale images
    temp = temp.reshape(32,32)
    plt.imshow(temp, cmap='gray')
  plt.show()


NameError: ignored

In [ ]:
  label_names = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
  conf = plot_confusion_matrix(make_knn(10, train_x, train_y), train_x, train_y, normalize='true',values_format  = '.2g', display_labels = label_names)
  conf.plot(ax=ax)
  conf.ax_.set_title("Validation Set Performance - Full Colour ")
  print("Time used on plot cnf: ", time.time()-starttime)

In [ ]:
#Transform RGB to gray values. Code from https://github.com/ppplinday/Picture-Classification-PCA-KNN/blob/master/KNN-PCA.py
def RGB_to_gray(data):
  data = data.reshape(data.shape[0],3,32,32).transpose(0,2,3,1).astype("float")
  newdata = np.zeros([data.shape[0], data.shape[1], data.shape[2]], float)
  num    = data.shape[0]
  height = data.shape[1]
  width  = data.shape[2]
  for n in range(num):
      for row in range(height):
          for col in range(width):
              newdata[n, row, col] = 0.299 * data[n, row, col, 0] + 0.587 * data[n, row, col, 1] + 0.114 * data[n, row, col, 2]
  newdata = newdata.reshape(data.shape[0], 32*32)
  return newdata


In [ ]:
#Transform into d components
def transform_to(d, X):
  starttime = time.time()
  p_pca = PCA(d)
  out = p_pca.fit_transform(X)
  print("Time used on making pca: ", time.time()-starttime)
  return out

In [ ]:
#Making the knn with k neighbours
def make_knn(k, X, Y):
  starttime = time.time()
  knn = KNeighborsClassifier(n_neighbors=k)
  knn = knn.fit(X, Y)
  print("Time used on making knn: ", time.time()-starttime)
  return knn

In [ ]:
#Calculating the accuracy of the knn model on a training set X with labels Y
def calc_accuracy(knn, X, Y):
  starttime = time.time()
  accuracy = knn.score(X,Y)
  #print("Accuracy on validation set with k = ", knn.n_neighbors , ": ", accuracy)
  print("Time used on calculating accuracy: ", time.time()-starttime)
  return accuracy

In [ ]:
#Makes a new sheet in the "accuracy"-workbook, with a name based on number of features and the length of the validation set.
def newSheet(d, val_x):
  wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1iqm61ROBjpCUysro7AoT0yrUd6z57VmVcPQ2FlyiHXA/edit#gid=0')
  title = "N:"+str(d)+".L:"+str(val_x.shape[0])
  rows = 100
  col = 2
  makeNew = True
  for ws in wb.worksheets():
    if (str(ws).split()[1]==str("'"+title+"'")):
      sheet = ws
      data = ws.get_all_values()
      makeNew = False
  if makeNew:
    sheet = wb.add_worksheet(title, rows, col)
    data = {'K':[0]*rows, 'Accuracy (%)':[0]*rows}
 
  df = pd.DataFrame(data)
  df.columns = df.iloc[0]
  df = df.iloc[1:]
  set_with_dataframe(sheet, df)
  return sheet, df

In [ ]:
#Calculating which k that gives the best accuracy on a data set val_x, when it is trained on train_x. These values are saved in a spreadsheet called "Accuracy".
def calculate_best_k(d, train_x, train_y, val_x, val_y):
  best_k = 1
  best_acc = 0.1
  
  sheet, df = newSheet(d, val_x)

  for k in range(1, 60):
    knn = make_knn(k, train_x, train_y)
    acc = calc_accuracy(knn, val_x, val_y)

    df.iloc[k-1][0]=k
    df.iloc[k-1][1]=acc*100
    print("k = ", k, " gir acc = ", acc,"\n")
    set_with_dataframe(sheet, df)

    if (acc>best_acc):
      best_k = k
      best_acc = acc
  return best_k, best_acc

In [ ]:
def pred(knn, X):
  starttime=time.time()
  out = knn.predict(X)
  print("Time used on predicting dataset: ", time.time()-starttime)
  return out

In [ ]:
#Viewing the reconstruction of the pca
def reconstruct_test():
  d = 165
  pca_i = PCA(d)
  pca_i.fit(x_train)
  x_train_pca = pca_i.transform(x_train)
  X_reconstructed_pca = pca_i.inverse_transform(x_train_pca)
  print(X_reconstructed_pca.shape)
  view(X_reconstructed_pca[100])



In [ ]:
def main_val_colour():
  #Load dataset
  dataset = load_data()
  print("Dataset loaded...")

  #Split dataset to training, validating and testing data.
  x_train, y_train, x_test, y_test = dataset['train_data'],dataset['train_labels'],dataset['test_data'],dataset['test_labels']
  train_x, train_y = x_train[0:45000],y_train[0:45000]
  val_x, val_y = x_train[45000:],y_train[45000:]
  mini_val_x, mini_val_y = val_x[500:], val_y[500:]
  print("Dataset splitted...")

  #Calculate best k by making kNNs an checking accuracy
  model = make_knn(1, train_x, train_y)
  #red_y = pred(model, val_x)
  #acc = accuracy_score(val_y, pred_y)
  #print("Model gives an accuracy of: ", acc)

  fig = plt.figure(figsize=[20, 20])
  ax = fig.add_subplot(2,1,1)

  starttime=time.time()
  label_names = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
  conf = plot_confusion_matrix(model, val_x, val_y, normalize='true',values_format  = '.2g', display_labels = label_names)
  conf.plot(ax=ax)
  conf.ax_.set_title("Validation Set Performance - Full Colour ")
  print("Time used on plot cnf: ", time.time()-starttime)

main_val_colour()


In [ ]:
def main_val_gray():
  #Load dataset
  dataset = load_data()
  print("Dataset loaded...")

  #Split dataset to training, validating and testing data.
  x_train, y_train, x_test, y_test = dataset['train_data'],dataset['train_labels'],dataset['test_data'],dataset['test_labels']
  train_x, train_y = x_train[0:45000],y_train[0:45000]
  val_x, val_y = x_train[45000:],y_train[45000:]
  mini_val_x, mini_val_y = val_x[500:], val_y[500:]
  print("Dataset splitted...")

  #Convert color images to gray images
  gray_train_x = RGB_to_gray(train_x)
  gray_val_x = RGB_to_gray(val_x) 
  print("Images converted...")
  
  #Calculate best k by making kNNs an checking accuracy
  model = make_knn(1, gray_train_x, train_y)
  #pred_y = pred(model, gray_val_x)
  #acc = accuracy_score(val_y, pred_y)
  #print("Model gives an accuracy of: ", acc)


  fig = plt.figure(figsize=[20, 20])
  ax = fig.add_subplot(2,1,1)

  starttime=time.time()
  label_names = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
  conf = plot_confusion_matrix(model, gray_val_x, val_y, normalize='true',values_format  = '.2g', display_labels = label_names)
  conf.plot(ax=ax)
  conf.ax_.set_title("Validation Set Performance - Full Gray ")
  print("Time used on plot cnf: ", time.time()-starttime)


main_val_gray()


In [ ]:
  fig = plt.figure(figsize=[20, 20])
  ax = fig.add_subplot(2,1,1) 
  model = make_knn(1,train_x, train_y)

  starttime=time.time()
  label_names = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
  conf = plot_confusion_matrix(model, train_x, train_y, normalize='true',values_format  = '.2g', display_labels = label_names)
  conf.plot(ax=ax)
  conf.ax_.set_title("Training Set Performance - Full Colour ")
  print("Time used on plot cnf: ", time.time()-starttime)

In [ ]:
def main_val_colour_pca():
  #Load dataset
  dataset = load_data()
  print("Dataset loaded...")

  #Split dataset to training, validating and testing data.
  x_train, y_train, x_test, y_test = dataset['train_data'],dataset['train_labels'],dataset['test_data'],dataset['test_labels']
  train_x, train_y = x_train[0:45000],y_train[0:45000]
  val_x, val_y = x_train[45000:],y_train[45000:]
  mini_val_x, mini_val_y = val_x[900:], val_y[900:]
  print("Dataset splitted...")

  

  #Find right number of components, according to a variance p
  p = 0.95
  #d = find_components(p, train_x)
  d = 215

  #Make PCA
  pca_train_x = transform_to(d, train_x)
  pca_val_x = transform_to(d, val_x) 

  
  #Calculate best k by making kNNs an checking accuracy
  model = make_knn(1, pca_train_x, train_y)
  pred_y = pred(model, pca_val_x)
  acc = accuracy_score(val_y, pred_y)
  print("Model gives an accuracy of: ", acc)

main_val_colour_pca()


In [ ]:
#Load dataset
dataset = load_data()
print("Dataset loaded...")

#Split dataset to training, validating and testing data.
x_train, y_train, x_test, y_test = dataset['train_data'],dataset['train_labels'],dataset['test_data'],dataset['test_labels']
train_x, train_y = x_train[0:45000],y_train[0:45000]
val_x, val_y = x_train[45000:],y_train[45000:]
mini_val_x, mini_val_y = val_x[900:], val_y[900:]
print("Dataset splitted...")
  

#Find right number of components, according to a variance p
p = 0.95
#d = find_components(p, train_x)
d = 215

#Make PCA
pca_train_x = transform_to(d, train_x)
pca_val_x = transform_to(d, val_x) 

  
#Calculate best k by making kNNs an checking accuracy
best_k=1
best_acc=0
accs=[]
for n in range(300, 301):
  model = make_knn(n, pca_train_x, train_y)
  pred_y = pred(model, pca_val_x)
  acc = accuracy_score(val_y, pred_y)
  accs.append(acc)
  print("Acc for ",n, " is ", acc)
  if (acc>best_acc):
    best_k=n
    best_acc=acc

print("Model gives an accuracy of: ", best_acc, " from k = ", best_k)

In [ ]:
def main_val_gray_pca():
  #Load dataset
  dataset = load_data()
  print("Dataset loaded...")

  #Split dataset to training, validating and testing data.
  x_train, y_train, x_test, y_test = dataset['train_data'],dataset['train_labels'],dataset['test_data'],dataset['test_labels']
  train_x, train_y = x_train[0:45000],y_train[0:45000]
  val_x, val_y = x_train[45000:],y_train[45000:]
  mini_val_x, mini_val_y = val_x[900:], val_y[900:]
  print("Dataset splitted...")

  #Convert color images to gray images
  gray_train_x = RGB_to_gray(train_x)
  gray_val_x = RGB_to_gray(val_x) 
  print("Images converted...")

  #Find right number of components, according to a variance p
  p = 0.95
  #d = find_components(p, gray_train_x)
  d = 159

  #Make PCA
  pca_train_x = transform_to(d, gray_train_x)
  pca_val_x = transform_to(d, gray_val_x) 

  #Calculate best k by making kNNs an checking accuracy
  model = make_knn(1, pca_train_x, train_y)
  pred_y = pred(model, pca_val_x)
  acc = accuracy_score(val_y, pred_y)
  print("Model gives an accuracy of: ", acc)

  fig = plt.figure(figsize=[20,20])
  ax = fig.add_subplot(1,1,1)
  conf = plot_confusion_matrix(model, pca_val_x, val_y, normalize='true', values_format='.2g')
  conf.plot(ax = ax)

main_val_gray_pca()


In [ ]:
#Load dataset
dataset = load_data()
print("Dataset loaded...")

#Split dataset to training, validating and testing data.
x_train, y_train, x_test, y_test = dataset['train_data'],dataset['train_labels'],dataset['test_data'],dataset['test_labels']
train_x, train_y = x_train[0:45000],y_train[0:45000]
val_x, val_y = x_train[45000:],y_train[45000:]
mini_val_x, mini_val_y = val_x[900:], val_y[900:]
print("Dataset splitted...")
  

#Find right number of components, according to a variance p
p = 0.95
#d = find_components(p, train_x)
d = 159

#Convert color images to gray images
gray_train_x = RGB_to_gray(train_x)
gray_val_x = RGB_to_gray(val_x)
gray_test_x = RGB_to_gray(x_test) 
print("Images converted...")

#Make PCA
pca_train_x = transform_to(d, gray_train_x)
pca_val_x = transform_to(d, gray_val_x) 
pca_test_x = transform_to(d, gray_test_x)

#starttime = time.time() 
#model = make_knn(100, pca_train_x, train_y)
#pred_y = pred(model, pca_test_x)
#acc = accuracy_score(y_test, pred_y)
#print("Acc for test set: ", acc, ". Done in ", time.time()-starttime)

#Calculate best k by making kNNs an checking accuracy
best_k=1
best_acc=0
accs=[]
for n in range (198, 231):
  model = make_knn(n, pca_train_x, train_y)
  pred_y = pred(model, pca_val_x)
  acc = accuracy_score(val_y, pred_y)
  accs.append(acc)
  print("Acc for ", n, " is ", acc)
if (acc>best_acc):
    best_k=n
    best_acc=acc

print("Accs: ", accs)
print("Model gives an accuracy of: ", best_acc, " from k = ", best_k)



In [ ]:
#Load dataset
dataset = load_data()
print("Dataset loaded...")

#Split dataset to training, validating and testing data.
x_train, y_train, x_test, y_test = dataset['train_data'],dataset['train_labels'],dataset['test_data'],dataset['test_labels']
train_x, train_y = x_train[0:45000],y_train[0:45000]
val_x, val_y = x_train[45000:],y_train[45000:]
mini_val_x, mini_val_y = val_x[900:], val_y[900:]
print("Dataset splitted...")
  

#Find right number of components, according to a variance p
p = 0.95
#d = find_components(p, train_x)
d = 159
d_col = 215

#Convert color images to gray images
gray_train_x = RGB_to_gray(train_x)
gray_val_x = RGB_to_gray(val_x)
gray_test_x = RGB_to_gray(x_test) 
print("Images converted...")

#Make PCA - gray
pca_train_x = transform_to(d, gray_train_x)
pca_val_x = transform_to(d, gray_val_x) 
pca_test_x = transform_to(d, gray_test_x)


#Make PCA - col
col_pca_train_x = transform_to(d_col, train_x)
col_pca_val_x = transform_to(d_col, val_x) 
col_pca_test_x = transform_to(d_col, x_test)


In [ ]:
for k in range(0,151, 10):
  print("Making for col...")

  #Colour
  starttime = time.time() 
  col_model = make_knn(k, col_pca_train_x, train_y)
  col_pred_y = pred(col_model, col_pca_test_x)
  col_acc = accuracy_score(y_test, col_pred_y)
  print("Coloured acc for test set with k = ", k, " is: ", col_acc, ". Done in ", time.time()-starttime)

  #Gray
  starttime = time.time() 
  model = make_knn(k, pca_train_x, train_y)
  pred_y = pred(model, pca_test_x)
  acc = accuracy_score(y_test, pred_y)
  print("Gray acc for test set with k = ", k, " is: ", acc, ". Done in ", time.time()-starttime)

In [ ]:
  fig = plt.figure(figsize=[20, 20])
  ax = fig.add_subplot(2,1,1)

  k_col = 120
  starttime=time.time()
  label_names = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
  col_model = make_knn(k_col, col_pca_train_x, train_y)
  conf = plot_confusion_matrix(col_model, col_pca_test_x, y_test, normalize='true',values_format  = '.2g', display_labels = label_names)
  conf.plot(ax=ax)
  conf.ax_.set_title("Test Set Performance - Colour with PCA ")
  print("Time used on plot cnf: ", time.time() - starttime)

  col_pred_y = pred(col_model, col_pca_test_x)
  col_acc = accuracy_score(y_test, col_pred_y)
  print("Coloured acc for test set with k = ", k_col, " is: ", col_acc, ". Done in ", time.time()-starttime)


In [ ]:
def main():
  #Load dataset
  dataset = load_data()
  print("Dataset loaded...")

  #Split dataset to training, validating and testing data.
  x_train, y_train, x_test, y_test = dataset['train_data'],dataset['train_labels'],dataset['test_data'],dataset['test_labels']
  train_x, train_y = x_train[0:49000],y_train[0:49000]
  val_x, val_y = x_train[49000:],y_train[49000:]
  mini_val_x, mini_val_y = val_x[900:], val_y[900:]
  print("Dataset splitted...")

  #Convert color images to gray images
  gray_train_x = RGB_to_gray(train_x)
  gray_val_x = RGB_to_gray(val_x) 
  print("Images converted...")

  #Find right number of components, according to a variance p
  p = 0.95
  d = find_components(p, gray_train_x)

  #Make PCA
  pca_train_x = transform_to(d, gray_train_x)
  pca_val_x = transform_to(d, gray_val_x) 

  #Calculate best k by making kNNs an checking accuracy
  best_k, best_acc = calculate_best_k(d, gray_train_x, train_y, gray_val_x, val_y)
  
  print("Best k is ", best_k, " with an accuracy of ", best_acc)
  

main()
